# Fertility Rate in Mexico 1950 to 2015

May 2019 - **@Quant_Girl**

This notebook contains a bokeh app showing the rapid change in Fertility Rate that took place in Mexico during the last 50 years. The idea is to illustrate how data visualisation tools can help to communicate a story.

Fertility rate in Mexico was close to 7 children during the 50s and 60s. It reached its peak of 6.83 children per woman in 1970. In contrast, the global average at that time was 4.73 and just above 2.2 for the UK and US. 



However, as data shows, something happened in the early 70s. This is the story behind the chart:

- In 1968, UN the United Nations declared family planning as a human right. Mexico voted in favor of the resolution, and not only in favor, but with deep conviction, in the hopes that women could achieve full equality under the law and in every facet of life. 

- In 1974 Mexico amended its Constitution (Article 4) to give people the right to decide when and if they want children and how many they want. 

- In 1977, a governmental coordinating body was made responsible for getting family planning services to all couples in need of them, and for reducing the growth rate to 2.5% annually by 1982.

- By December 1978 an estimated 1.7 million active contraceptive users were enrolled in government sponsored family planning programs -- 19% of all married women of reproductive age. Another estimated 17% were obtaining modern contraceptive services and supplies through nongovernment channels, and a further 4% were using rhythm, withdrawal or folk methods.

- Fertility rate decreased steadily until the point of becoming smaller than the global average in 2003 and reaching its current level of around 2.2 children per woman (comparable to developed contries like US and UK).

In [1]:
import pandas as pd
from bokeh.io import curdoc, output_notebook, push_notebook, show
from bokeh.layouts import column, row, widgetbox
from bokeh.models import ColumnDataSource, Slider, Button, HoverTool, WidgetBox, Div, Panel, Paragraph
from bokeh.plotting import figure
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
output_notebook()

Loading BokehJS ...

In [2]:
data_un = pd.read_csv('Data/children-per-woman-UN.csv') #Source: https://population.un.org/wpp/Download/Standard/Population/

In [3]:
#Bokeh App
data_un.columns = ['Country', 'Code', 'Year', 'Fertility']

def modify_doc(doc):
    
    def get_data(df=data_un, countries = ['Mexico', 'World', 'United Kingdom', 'United States'], year = 1950):
    
        data = df[(df.Country.isin(countries))&(df.Year <=year)].reset_index(drop=True)
        aux = data.Country
        data['Color'] = aux.map({'Mexico':'green', 'World':'orange', 'United Kingdom':'red', 'United States':'blue'})

        return ColumnDataSource(data)

    def get_plot(src):

        plot = figure(plot_height = 600, plot_width = 800,
                  x_range = (1950, 2016), y_range = (0, 8),
                  x_axis_label = 'Year',
                  y_axis_label = 'Fertility Rate (children per woman)',
                  tools = "crosshair,pan, save,wheel_zoom, reset")

        plot.circle(x = 'Year', y = 'Fertility', color ='Color', legend = 'Country', fill_alpha = 0.75, size = 5, source = src, )

        hover = HoverTool(tooltips = [
        ('Year', '@Year'),
        ('Country', '@Country'),
        ('Fertility Average', '@Fertility{(0.00)}')])
        plot.add_tools(hover)

        return(plot)
    
    def animate_update():
        
        year = year_select.value + 1
        if year > 2015:
            year = 1950
            
        year_select.value = year 
    

    def slider_update(attr, old, new):

        year_end = year_select.value
        new_data = get_data(year = year_end)
        src.data.update(new_data.data)
        
        
    year_select = Slider(start = 1950, end = 2015, value = 1970, step = 1, title = 'Year')
    year_select.on_change('value', slider_update)
    callback_id = None

    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = curdoc().add_periodic_callback(animate_update, 200)
        else:
            button.label = '► Play'
            curdoc().remove_periodic_callback(callback_id)    
    

    button = Button(label='► Play', width=60)
    button.on_click(animate)

    
    src = get_data(data_un, year=1970)
    p = get_plot(src)
    controls = WidgetBox(year_select, button)
    
    title = Div(text='<h1 style="text-align: left">Fertility Rate in Mexico 1950 - 2015</h1>', width =700)
    blank = Div(text='<h1 style="text-align: left"> </h1>', width =300)
    
    par1 = Paragraph(text = """ 
                    The number of births per Mexican woman during her childbearing years (the total fertility rate, or simply fertility rate) 
                    was close to 7 children during the 50s and 60s. It reached its peak of 6.83 children per woman in 1970. 
                    In contrast, the global average at that time was 4.73 and just above 2.2 for the UK and US.""", width =300)
    
    par2 = Paragraph(text = """
                    This visualisation (Press '► Play' button below) shows the rapid change that took place in Mexico during the last 50 years.
                    Fertility rate decreased steadily until the point
                    of becoming smaller than the global average in 2003 and reaching its current level of around 2.2.""", width =300)
    
    author = Paragraph(text = """Python Code: @QuantGirl""", width =300)        
    
    source = Paragraph(text = """
                    Data Source: https://population.un.org/wpp/Download/Standard/Population/""", width =300)    
    
    signature = Paragraph(text = """@QuantGirl""")
    
    layout = column(title, row(p, column(par1, par2, author, source, controls)), signature ) 

    doc.add_root(layout)
    


handler = FunctionHandler(modify_doc)
app = Application(handler)



In [4]:
show(app)